# Log metrics with TensorBoard in PyTorch Lightning

description: log tensorboard metrics with pytorch lightning and visualize metrics in tensorboard

Lightning supports many popular [logging frameworks](https://pytorch-lightning.readthedocs.io/en/stable/loggers.html). In this tutorial we will go over using the built-in TensorBoard logger and leveraging Azure ML's TensorBoard integration to visualize the metrics.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
# training script
source_dir = "src"
script_name = "train-with-tensorboard-logging.py"

# environment file
environment_file = "environment.yml"

# azure ml settings
environment_name = "pt-lightning"
experiment_name = "pt-lightning-tensorboard-tutorial"
compute_name = "gpu-K80-2"

## Create environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment. This notebook will use the same environment definition that was used for part 1 of the tutorial. Note that TensorBoard is the default logger in Lightning and comes preinstalled, so you don't need to add the **tensorboard** package as a dependency to the environment for the remote job.


In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(environment_name, environment_file)

# specify a GPU base image
env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04"
)

## Enable logging in training script

In *train_with_tensorboard_logging.py*:

### 1. Specify location to write logs
Specify the path of the location for the logger to write logs out to. In this tutorial, we add a `--logdir` argument to the training script, with the default value of `./logs`. This is the path you should write your TensorBoard logs out to if you would like to use the Azure ML TensorBoard integration (see the following section). You can override the default value if you wish to write logs to a different path.

### 2. Create a TensorBoardLogger
Create a `TensorBoardLogger` in your training script and pass it to the `logger` parameter of the `Trainer()` call.

```python
    tb_logger = TensorBoardLogger(args.logdir)
    trainer = pl.Trainer.from_argparse_args(args, logger=tb_logger)
```
    
### 3. Log metrics
You can then log metrics and other objects in your script. In this tutorial's training script, we leverage Lightning's automatic log functionalities to log the loss metric by calling `self.log()` inside the `training_step()` method.


For more information on logging and the configurable options, see Lightning's [Logging](https://pytorch-lightning.readthedocs.io/en/stable/logging.html) documentation and the [TensorBoardLogger](https://pytorch-lightning.readthedocs.io/en/stable/logging.html#tensorboard) reference documentation.

## Configure and run training job

Create a ScriptRunConfig to specify the training script & arguments, environment, and cluster to run on.

In [ ]:
from azureml.core import ScriptRunConfig, Experiment

src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=[
        "--max_epochs",
        25,
        "--gpus",
        2,
        "--accelerator",
        "ddp",
        "--logdir",
        "./logs",
    ],
    compute_target=compute_name,
    environment=env,
)

run = Experiment(ws, experiment_name).submit(src)
run

## Visualize logs in TensorBoard

Azure ML provides an integration for users to easily stream and visualize the logs from their remote job in TensorBoard. To use this functionality, make sure you have the **azureml-tensorboard** and **tensorflow** packages on your machine where you are running this notebook.

You can launch TensorBoard either during your job or after it completes. First, create an Azure ML TensorBoard object and pass it the run(s) with the logs you wish to visualize. The TensorBoard constructor takes an array of runs, so if you only want to visualize one run, pass it in as a single-element array.

Then, call the `start()` method, which will launch and start the TensorBoard server on your local machine. This will give you the URI from where you can access TensorBoard in your browser. By default this will be http://localhost:6006/. You can change the port that the TensorBoard instance will run on by specifying the `port` parameter to the `TensorBoard()` call.

In [ ]:
from azureml.tensorboard import Tensorboard

tb = Tensorboard([run])

# If successful, start() returns a string with the URI of the instance.
tb.start()

After you are done using TensorBoard, be sure to call `stop()` to stop the TensorBoard instance, otherwise it will continue to run until you shut down your notebook kernel.

In [ ]:
tb.stop()

For more information on using TensorBoard to visualize your Azure ML experiments, see [Visualize metrics with TensorBoard](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-tensorboard) and the [TensorBoard reference documentation](https://docs.microsoft.com/python/api/azureml-tensorboard/azureml.tensorboard.tensorboard?view=azure-ml-py).

In [ ]:
run.wait_for_completion(show_output=True)